# Implementing a Custom LangGraph Agent for ARE

This notebook demonstrates how to implement a custom agent using LangGraph that works with the ARE framework.

## What You'll Learn
1. How to implement the `RunnableARESimulationAgent` interface
2. Converting ARE tools to LangChain format
3. Handling notifications from the environment
4. Running your custom agent on scenarios
5. Analyzing agent performance

In [ ]:
# Install required packages
# !pip install meta-agents-research-environments langgraph langchain-openai

In [ ]:
import sys
import os
sys.path.insert(0, '../meta-agents-research-environments')

from typing import Any, Callable
from datetime import datetime
import json

# ARE imports
from are.simulation.agents.are_simulation_agent import RunnableARESimulationAgent
from are.simulation.agents.agent_execution_result import AgentExecutionResult
from are.simulation.agents.agent_log import ActionLog, BaseAgentLog
from are.simulation.scenarios.scenario import Scenario
from are.simulation.notification_system import BaseNotificationSystem, Message, MessageType
from are.simulation.environment import Environment, EnvironmentConfig
from are.simulation.scenarios.registration import get_scenario_by_id

# LangGraph imports
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage

## 1. Implementing the Custom Agent Class

We'll create a LangGraph-based agent that implements the ARE agent interface.

In [ ]:
class LangGraphAREAgent(RunnableARESimulationAgent):
    """Custom agent using LangGraph ReAct framework for ARE."""
    
    def __init__(self, model_name: str = "gpt-4", api_key: str | None = None):
        """
        Initialize the LangGraph agent.
        
        Args:
            model_name: OpenAI model name
            api_key: OpenAI API key (or set OPENAI_API_KEY env var)
        """
        self.model_name = model_name
        
        if api_key:
            os.environ['OPENAI_API_KEY'] = api_key
        
        self.llm = ChatOpenAI(model=model_name, temperature=0)
        self.agent = None
        self.conversation_history = []
        self.tools = []
        self.agent_logs = []
        
    def run_scenario(
        self,
        scenario: Scenario,
        notification_system: BaseNotificationSystem | None,
        initial_agent_logs: list[BaseAgentLog] | None = None,
    ) -> AgentExecutionResult:
        """
        Main entry point for running the agent on a scenario.
        
        Args:
            scenario: The scenario to run
            notification_system: System for receiving environment notifications
            initial_agent_logs: Previous execution logs (for replay)
        
        Returns:
            AgentExecutionResult with execution details
        """
        print(f"\n{'='*80}")
        print(f"Running LangGraph Agent on Scenario: {scenario.scenario_id}")
        print(f"{'='*80}\n")
        
        # Initialize agent logs
        self.agent_logs = initial_agent_logs or []
        
        # Convert ARE tools to LangChain format
        self.tools = self._convert_are_tools_to_langchain(scenario.get_tools())
        print(f"Loaded {len(self.tools)} tools")
        
        # Create LangGraph ReAct agent
        self.agent = create_react_agent(
            self.llm, 
            self.tools,
            state_modifier="You are a helpful AI assistant. Use the available tools to complete the task."
        )
        
        # Subscribe to notifications
        if notification_system:
            self._setup_notification_handler(notification_system)
        
        # Run the agent loop
        result = self._run_agent_loop(notification_system)
        
        return result
    
    def _convert_are_tools_to_langchain(self, are_tools):
        """
        Convert ARE tools to LangChain tool format.
        
        ARE tools have:
        - name: Unique identifier
        - function: Callable
        - function_description: Description
        - parameters_schema: JSON schema
        """
        langchain_tools = []
        
        for are_tool in are_tools:
            # Create a wrapper function that logs the call
            original_func = are_tool.function
            
            def make_wrapper(func, tool_name):
                def wrapper(*args, **kwargs):
                    print(f"\n🔧 Calling tool: {tool_name}")
                    print(f"   Args: {kwargs}")
                    
                    result = func(**kwargs)
                    
                    print(f"   Result: {str(result)[:200]}...")
                    return result
                return wrapper
            
            # Create LangChain tool
            lc_tool = tool(
                name=are_tool.name,
                description=are_tool.function_description
            )(make_wrapper(original_func, are_tool.name))
            
            langchain_tools.append(lc_tool)
        
        return langchain_tools
    
    def _setup_notification_handler(self, notification_system: BaseNotificationSystem):
        """Set up handler for environment notifications."""
        
        def handle_notification(message: Message):
            """Handle incoming notifications from environment."""
            print(f"\n📬 Notification: {message.message_type.value}")
            print(f"   Content: {message.message}")
            
            # Add to conversation history if it's a message to the agent
            if message.message_type == MessageType.USER_MESSAGE:
                self.conversation_history.append(
                    HumanMessage(content=message.message)
                )
        
        notification_system.subscribe(handle_notification)
    
    def _run_agent_loop(self, notification_system: BaseNotificationSystem | None) -> AgentExecutionResult:
        """
        Run the main agent loop.
        
        This simplified version demonstrates the concept.
        In practice, you'd need to handle the message queue properly.
        """
        start_time = datetime.now()
        
        # Wait for initial message from environment
        # In a real implementation, you'd poll the notification system
        # For demo purposes, we'll simulate waiting
        import time
        time.sleep(2)
        
        # Check if we received a task message
        if not self.conversation_history:
            # No message received, create a dummy one for demo
            task = "Please help me complete the task."
            self.conversation_history.append(HumanMessage(content=task))
        
        print(f"\n💬 Task: {self.conversation_history[-1].content}\n")
        
        # Run the agent
        try:
            response = self.agent.invoke(
                {"messages": self.conversation_history}
            )
            
            # Extract final response
            final_message = response["messages"][-1]
            final_response = final_message.content
            
            print(f"\n✅ Agent Response: {final_response}\n")
            
            # Calculate metrics
            execution_time = (datetime.now() - start_time).total_seconds()
            num_steps = len(response["messages"]) // 2  # Rough estimate
            
            return AgentExecutionResult(
                success=True,
                final_response=final_response,
                num_steps=num_steps,
                execution_time=execution_time
            )
            
        except Exception as e:
            print(f"\n❌ Error: {e}\n")
            execution_time = (datetime.now() - start_time).total_seconds()
            
            return AgentExecutionResult(
                success=False,
                final_response=str(e),
                num_steps=0,
                execution_time=execution_time
            )
    
    def stop(self):
        """Stop the agent (optional)."""
        print("Stopping agent...")
        # Cleanup if needed

## 2. Testing the Custom Agent

Let's test our custom agent on a simple scenario.

In [ ]:
# Set your OpenAI API key
# Option 1: Set as environment variable
# os.environ['OPENAI_API_KEY'] = 'your-key-here'

# Option 2: Pass to agent constructor
# agent = LangGraphAREAgent(model_name="gpt-4", api_key="your-key-here")

# For demo, we'll use a mock (set provider to 'mock' in environment config)
agent = LangGraphAREAgent(model_name="gpt-4")

In [ ]:
# Load a scenario
scenario = get_scenario_by_id("scenario_find_image_file")
scenario.initialize()

print(f"Scenario: {scenario.scenario_id}")
print(f"Apps: {[app.name for app in scenario.apps or []]}")
print(f"Tools: {len(scenario.get_tools())}")

In [ ]:
# Run the agent on the scenario
# Note: This is a simplified demo. In practice, you'd integrate with
# the environment's event loop and notification system properly.

from are.simulation.notification_system import NotificationSystem

notification_system = NotificationSystem()

result = agent.run_scenario(
    scenario=scenario,
    notification_system=notification_system
)

print(f"\n{'='*80}")
print("Execution Result:")
print(f"{'='*80}")
print(f"Success: {result.success}")
print(f"Steps: {result.num_steps}")
print(f"Time: {result.execution_time:.2f}s")
print(f"Response: {result.final_response}")

## 3. Integrating with ARE Environment

For full integration, you need to register your agent with the ARE CLI.

Create a file `my_custom_agents.py`:

In [ ]:
integration_code = '''
# my_custom_agents.py
from are.simulation.agents.agent_builder import AbstractAgentBuilder
from are.simulation.agents.are_simulation_agent import RunnableARESimulationAgent

class MyAgentBuilder(AbstractAgentBuilder):
    """Custom agent builder for LangGraph agent."""
    
    def list_agents(self) -> list[str]:
        """List available agents."""
        return ["langgraph-react"]
    
    def build(
        self,
        agent_name: str,
        **kwargs
    ) -> RunnableARESimulationAgent:
        """Build and return agent instance."""
        if agent_name == "langgraph-react":
            # Import your agent class
            from my_agent_module import LangGraphAREAgent
            return LangGraphAREAgent(
                model_name=kwargs.get("model", "gpt-4")
            )
        else:
            raise ValueError(f"Unknown agent: {agent_name}")

# Then use it with ARE CLI:
# are-run -s scenario_find_image_file --agent langgraph-react --model gpt-4
'''

print(integration_code)

## 4. Advanced: Handling Real-Time Notifications

For scenarios with dynamic events, you need to handle notifications properly.

In [ ]:
class AdvancedLangGraphAgent(LangGraphAREAgent):
    """LangGraph agent with proper notification handling."""
    
    def __init__(self, model_name: str = "gpt-4", api_key: str | None = None):
        super().__init__(model_name, api_key)
        self.pending_notifications = []
        self.is_running = False
    
    def _setup_notification_handler(self, notification_system: BaseNotificationSystem):
        """Enhanced notification handler that queues messages."""
        
        def handle_notification(message: Message):
            print(f"\n📬 Received: {message.message_type.value}")
            
            if message.message_type == MessageType.USER_MESSAGE:
                # Queue the message for processing
                self.pending_notifications.append(message)
                print(f"   Queued message: {message.message[:100]}...")
            
            elif message.message_type == MessageType.ENVIRONMENT_UPDATE:
                # Handle environment changes
                print(f"   Environment changed: {message.message}")
                # You might want to update agent context here
            
            elif message.message_type == MessageType.ENVIRONMENT_STOP:
                # Environment is stopping
                self.is_running = False
                print("   Environment stopped")
        
        notification_system.subscribe(handle_notification)
    
    def _run_agent_loop(self, notification_system: BaseNotificationSystem | None):
        """Enhanced agent loop that processes notifications."""
        start_time = datetime.now()
        self.is_running = True
        
        # Wait for initial task message
        import time
        max_wait = 10  # seconds
        waited = 0
        
        while not self.pending_notifications and waited < max_wait:
            time.sleep(0.5)
            waited += 0.5
        
        if not self.pending_notifications:
            print("⚠️ No task received within timeout")
            return AgentExecutionResult(
                success=False,
                final_response="No task received",
                num_steps=0,
                execution_time=(datetime.now() - start_time).total_seconds()
            )
        
        # Process each notification/task
        all_responses = []
        
        while self.is_running and self.pending_notifications:
            notification = self.pending_notifications.pop(0)
            
            print(f"\n💬 Processing task: {notification.message[:100]}...\n")
            
            # Add to conversation
            self.conversation_history.append(
                HumanMessage(content=notification.message)
            )
            
            # Run agent on this message
            try:
                response = self.agent.invoke(
                    {"messages": self.conversation_history}
                )
                
                final_message = response["messages"][-1]
                all_responses.append(final_message.content)
                
                # Update conversation history
                self.conversation_history = response["messages"]
                
                print(f"\n✅ Response: {final_message.content}\n")
                
            except Exception as e:
                print(f"\n❌ Error: {e}\n")
                all_responses.append(f"Error: {e}")
        
        execution_time = (datetime.now() - start_time).total_seconds()
        
        return AgentExecutionResult(
            success=True,
            final_response="\n\n".join(all_responses),
            num_steps=len(self.conversation_history) // 2,
            execution_time=execution_time
        )

## Next Steps

1. **Add Memory**: See `03_agent_with_memory.ipynb` for memory-augmented agents
2. **Run Benchmarks**: Test on GAIA2 validation set
3. **Optimize**: Profile and improve agent performance
4. **Experiment**: Try different prompting strategies and tool usage patterns

## Key Points

- Implement `RunnableARESimulationAgent` interface
- Convert ARE tools to your framework's format
- Handle notifications for dynamic scenarios
- Return `AgentExecutionResult` with metrics
- Register with ARE CLI for full integration